In [22]:
import numpy as np
import pandas as pd
from itertools import product
from math import sqrt

import gurobipy as gp
from gurobipy import GRB

# Read excel file

In [23]:
filename = 'input_data.xlsx'
data = pd.read_excel(filename,header = None)
data.head()
#sort the data
patients_demand = data[[0,1,2]]
existing_hospitals = data[[3,4,5]]
temporary_facilities = data[[6,7,8]]
data.head()

,0,1,2,3,4,5,6,7,8,9,10
0,DEMAND,NaN,NaN,EXISTING CAPACITY,NaN,NaN,TEMPORARY FACILITIES,NaN,NaN,Cost of building a new facility,500000.0
1,ABS,Coordinates,Demand,Hospital,Coordinates,Capacity,Facility name,Coordinates,Capacity,Cost of driving the patients / 10km,15.0
2,CAP RAMBLA (Terrassa),"41.5624, 2.0109",20,Hospital Terrassa,"41.5641, 2.0519",50,Pavelló Can Jofresa (Terrassa),"41.5490, 2.0218",30,NaN,NaN
3,CAP GRÀCIA (Sabadell),"41.5422, 2.1060",12,Hospital Taulí,"41.5589, 2.1109",35,Club Natació Sabadell,"41.5627, 2.1020",30,NaN,NaN
4,CAP BARCELONETA (Barcelona),"41.3830, 2.1944",2,Hospital del Mar,"41.3906, 2.1927",20,Pavelló Olímpic Badalona,"41.4429, 2.2332",40,NaN,NaN


In [24]:
def create_dictionary(df):
    multidata = dict()
    for index,row in df.iterrows():
        multidata[index] = [row[1],row[2]]
    return multidata

### 1. Demand of patients by ABS

In [25]:
new_header = patients_demand.iloc[1] #grab the first row for the header
patients_demand = patients_demand[2:] #take the data less the header row
patients_demand.columns = new_header #set the header row as the df header
#demand = demand.set_index(['ABS'])
patients_demand = patients_demand.reset_index(drop = True)
patients_demand = patients_demand.dropna()
patients_demand.head()

1,ABS,Coordinates,Demand
0,CAP RAMBLA (Terrassa),"41.5624, 2.0109",20
1,CAP GRÀCIA (Sabadell),"41.5422, 2.1060",12
2,CAP BARCELONETA (Barcelona),"41.3830, 2.1944",2
3,CAP PASSEIG MARAGALL (Barcelona),"41.4161, 2.1813",25
4,CAP EL GUINARDO (Barcelona),"41.4202, 2.1788",10


In [26]:
dictionary_demand = create_dictionary(patients_demand)
dictionary_demand

{0: ['41.5624, 2.0109', 20],
 1: ['41.5422, 2.1060', 12],
 2: ['41.3830, 2.1944', 2],
 3: ['41.4161, 2.1813', 25],
 4: ['41.4202, 2.1788', 10],
 5: ['41.4112, 2.1575', 20],
 6: ['41.5624, 2.1065', 30],
 7: ['41.5344, 2.1189', 50],
 8: ['41.5651, 2.0173', 10],
 9: ['41.5621, 1.9846', 25],
 10: ['41.3691, 2.1313', 40],
 11: ['41.3762, 2.1169', 10],
 12: ['41.3591, 2.1161', 10],
 13: ['41.4376, 2.2339', 4],
 14: ['41.4465, 2.2318', 10],
 15: ['41.4617, 2.2416', 35]}

In [27]:
dictionary_name_abs = patients_demand.ABS.to_dict()
dictionary_name_abs

{0: 'CAP RAMBLA (Terrassa)',
 1: 'CAP GRÀCIA (Sabadell)',
 2: 'CAP BARCELONETA (Barcelona)',
 3: 'CAP PASSEIG MARAGALL (Barcelona)',
 4: 'CAP EL GUINARDO (Barcelona)',
 5: 'CAP LARRARD (Barcelona)',
 6: 'CAP CREU ALTA (Sabadell)',
 7: 'CAP SANT FÈLIX (Sabadell)',
 8: 'CAP SANT LLÀTZER (Terrassa)',
 9: 'CAP CAN TRIES (Terrassa)',
 10: 'CAP SANTA EULÀLIA (Hospitalet)',
 11: 'CAP COLLBLANC (Hospitalet)',
 12: 'CAP GORNAL (Hospitalet)',
 13: 'CAP SANT ROC (Badalona)',
 14: 'CAP LA SALUT (Badalona)',
 15: 'CAP BUFALÀ (Badalona)'}

### 2. Capacity of existing hospitals

In [28]:
new_header = existing_hospitals.iloc[1] #grab the first row for the header
existing_hospitals = existing_hospitals[2:] #take the data less the header row
existing_hospitals.columns = new_header #set the header row as the df header
#existing_hospitals = existing_hospitals.set_index(['Hospital'])
existing_hospitals = existing_hospitals.reset_index(drop = True)
existing_hospitals = existing_hospitals.dropna()
existing_hospitals.head()

1,Hospital,Coordinates,Capacity
0,Hospital Terrassa,"41.5641, 2.0519",50
1,Hospital Taulí,"41.5589, 2.1109",35
2,Hospital del Mar,"41.3906, 2.1927",20
3,Hospital de Barcelona,"41.5614, 2.0520",12
4,Hospital Badalona,"41.5589, 2.1109",30


In [29]:
dictionary_existing_hospitals = create_dictionary(existing_hospitals)
dictionary_existing_hospitals

{0: ['41.5641, 2.0519', 50],
 1: ['41.5589, 2.1109', 35],
 2: ['41.3906, 2.1927', 20],
 3: ['41.5614, 2.0520', 12],
 4: ['41.5589, 2.1109', 30]}

In [30]:
dictionary_name_facilities = existing_hospitals.Hospital.to_dict()

### 3. Possible new facilities

In [31]:
new_header = temporary_facilities.iloc[1] #grab the first row for the header
temporary_facilities = temporary_facilities[2:] #take the data less the header row
temporary_facilities.columns = new_header #set the header row as the df header
temporary_facilities = temporary_facilities.dropna()
indices = [*range(len(dictionary_existing_hospitals),len(dictionary_existing_hospitals)+len(temporary_facilities))]
temporary_facilities.index = indices

temporary_facilities.head()

1,Facility name,Coordinates,Capacity
5,Pavelló Can Jofresa (Terrassa),"41.5490, 2.0218",30
6,Club Natació Sabadell,"41.5627, 2.1020",30
7,Pavelló Olímpic Badalona,"41.4429, 2.2332",40
8,Hotel Marina Barcelona,"41.3941, 2.1922",60
9,Fira de Barcelona,"41.3745, 2.1529",200


In [32]:
temporary_facilities_dictionary = create_dictionary(temporary_facilities)
temporary_facilities_dictionary

{5: ['41.5490, 2.0218', 30],
 6: ['41.5627, 2.1020', 30],
 7: ['41.4429, 2.2332', 40],
 8: ['41.3941, 2.1922', 60],
 9: ['41.3745, 2.1529', 200]}

In [33]:
dictionary_name_facilities = {**dictionary_name_facilities,**temporary_facilities['Facility name'].to_dict()} #temporary_facilities['Facility name'].to_dict()
dictionary_name_facilities

{0: 'Hospital Terrassa',
 1: 'Hospital Taulí',
 2: 'Hospital del Mar',
 3: 'Hospital de Barcelona',
 4: 'Hospital Badalona',
 5: 'Pavelló Can Jofresa (Terrassa)',
 6: 'Club Natació Sabadell',
 7: 'Pavelló Olímpic Badalona',
 8: 'Hotel Marina Barcelona',
 9: 'Fira de Barcelona'}

### 4. Cost of building a new facility and cost per 10km to drive patients

In [34]:
cost_new_facility =  data.iloc[0,10]
cost_drive_patients = data.iloc[1,10]

##  Definition of help functions: distance computations and solver

In [35]:
def compute_distance(loc1, loc2):
    # This function determines the Euclidean distance between a facility and an ABS centroid.
    loc1= loc1[1:-1]
    loc2= loc2[1:-1]
    loc1 = [float(x.strip()) for x in loc1.split(',')]
    loc2 = [float(x.strip()) for x in loc2.split(',')]
    dx = loc1[0] - loc2[0]
    dy = loc1[1] - loc2[1]
    return sqrt(dx*dx + dy*dy)

In [36]:
# Create a dictionary to capture the coordinates of an ABS and the demand of COVID-19 treatment
ABS, c_coordinates, demand  = gp.multidict(dictionary_demand)

## Setting parameters

In [37]:
# Indices for the ABS
ABS = [*range(0,len(dictionary_demand))]
# Indices for the facilities
facilities = [*range(0,len(dictionary_existing_hospitals)+len(temporary_facilities_dictionary))]

#existing facility and capacity of treating COVID-19 patients
existing, e_coordinates, e_capacity  = gp.multidict(dictionary_existing_hospitals)

#temporary facility and capacity of treating COVID-19 patients
temporary, t_coordinates, t_capacity  = gp.multidict(temporary_facilities_dictionary)

# Cost of driving 10 km
dcost = cost_drive_patients
# Cost of building a temporary facility with capacity of COVID-19
tfcost = cost_new_facility

# Compute key parameters of MIP model formulation
f_coordinates = {}
for e in existing:
    f_coordinates[e] = e_coordinates[e]

for t in temporary:
    f_coordinates[t] = t_coordinates[t]

# Cartesian product of ABS and facilities
cf = []

for c in ABS:
    for f in facilities:
        tp = c,f
        cf.append(tp)

# Compute distances between ABS centroids and facility locations
distance = {(c,f): compute_distance(c_coordinates[c], f_coordinates[f]) for c, f in cf}

## Open output file

In [38]:
import sys

orig_stdout = sys.stdout
f = open('output_temporary_facilities.txt', 'w')
sys.stdout = f

## Model MIP

### Objective function: Minimize total distance to drive to a COVID-19 facility
### ABS demand constraints
### Existing facilities capacity constraints
### temporary facilities capacity constraints

In [39]:
m = gp.Model('FACILITY_LOCATION_COVID19')

# Build temporary facility
y = m.addVars(temporary, vtype=GRB.BINARY, name='temporary')

# Assign COVID-19 patients of ABS to facility
x = m.addVars(cf, vtype=GRB.CONTINUOUS, name='Assign')

# Add capacity to temporary facilities
z = m.addVars(temporary, vtype=GRB.CONTINUOUS, name='addCap')

# Objective function: Minimize total distance to drive to a COVID-19 facility
# Big penalty for adding capacity at a temporary facility
penaltyC = 1e9
m.setObjective(gp.quicksum(dcost*distance[c,f]*x[c,f] for c,f in cf) 
               + tfcost*y.sum()
               + penaltyC*z.sum(), GRB.MINIMIZE)

# ABS demand constraints
demandConstrs = m.addConstrs((gp.quicksum(x[c,f] for f in facilities) == demand[c] for c in ABS), 
                             name='demandConstrs')

# Existing facilities capacity constraints
existingCapConstrs = m.addConstrs((gp.quicksum(x[c,e]  for c in ABS) <= e_capacity[e] for e in existing ), 
                                  name='existingCapConstrs')

# temporary facilities capacity constraints
temporaryCapConstrs = m.addConstrs((gp.quicksum(x[c,t]  for c in ABS) -z[t] 
                                    <= t_capacity[t]*y[t] for t in temporary ),
                                   name='temporaryCapConstrs')
# Run optimization engine
m.optimize()

In [40]:
# Total cost of building temporary facility locations
temporary_facility_cost = 0

print(f"\n\n_____________Optimal costs______________________")
for t in temporary:
    if (y[t].x > 0.5):
        temporary_facility_cost += tfcost*round(y[t].x)

patient_allocation_cost = 0
for c,f in cf:
    if x[c,f].x > 1e-6:
        patient_allocation_cost += dcost*round(distance[c,f]*x[c,f].x)

print(f"The total cost of building COVID-19 temporary healthcare facilities is €{temporary_facility_cost:,}") 
print(f"The total cost of allocating COVID-19 patients to healthcare facilities is ${patient_allocation_cost:,}")  

# Build temporary facility at location

print(f"\n_____________Plan for temporary facilities______________________")
for t in temporary:
    if (y[t].x > 0.5):
        #print(f"Build a temporary facility at location {t}")
        t = dictionary_name_facilities[t]
        print(f"Build a temporary facility at location {t}")

# Extra capacity at temporary facilities
print(f"\n_____________Plan to increase Capacity at temporary Facilities______________________")
for t in temporary:
    if (z[t].x > 1e-6):
        t = dictionary_name_facilities[t]
        print(f"Increase  temporary facility capacity at location {t} by {round(z[t].x)} beds")

# Demand satisfied at each facility
f_demand = {}

print(f"\n_____________Allocation of ABS patients to COVID-19 healthcare facility______________________")
for f in facilities:
    temp = 0
    for c in ABS:
        allocation = round(x[c,f].x)
        if allocation > 0:
            f_ = dictionary_name_facilities[f]
            c_ = dictionary_name_abs[c]
            print(f"{allocation} COVID-19 patients from {c_} are treated at facility {f_} ") #FROM ABS
        temp += allocation
    f_demand[f] = temp
    
    if temp > 0:
        print(f"{temp} is the total number of COVID-19 patients that are treated at facility {f_}. ")
        print(f"\n________________________________________________________________________________")

# Test total demand = total demand satisfied by facilities
total_demand = 0

for c in ABS:
    total_demand += demand[c]

demand_satisfied = 0
for f in facilities:
    demand_satisfied += f_demand[f]

print(f"\n_____________Test demand = supply______________________")
print(f"Total demand is: {total_demand:,} patients")
print(f"Total demand satisfied is: {demand_satisfied:,} beds")

##  References
[1] Katherine Klise and Michael Bynum. *Facility Location Optimization Model for COVID-19 Resources*. April 2020. Joint DOE Laboratory Pandemic Modeling and Analysis Capability. SAND2020-4693R.